# Few Shot: JMS -> SmallRye Reactive - jms-to-reactive-quarkus-00010

> Work with a LLM to generate a fix for the rule: jms-to-reactive-quarkus-00010

We will include the solved example in this example to see if we get better quality results

##### Sample Applications Used
* 2 sample apps from [JBoss EAP Quickstarts](https://github.com/jboss-developer/jboss-eap-quickstarts/tree/7.4.x) were chosen:  helloworld-mdb & cmt
    * [helloworld-mdb](https://github.com/savitharaghunathan/helloworld-mdb)
    * [cmt](https://github.com/konveyor-ecosystem/cmt)

##### Using Custom Rules for JMS to SmallRye Reactive
* Rules were developed by Juanma [@jmle](https://github.com/jmle)
    * Rules originally from: https://github.com/jmle/rulesets/blob/jms-rule/default/generated/quarkus/05-jms-to-reactive-quarkus.windup.yaml


In [67]:
# Install local kai package in the current Jupyter kernel
import sys

!{sys.executable} -m pip install -e ../../

Obtaining file:///Users/sraghuna/local_dev/kai
  Preparing metadata (setup.py) ... done
  Attempting uninstall: kai
    Found existing installation: kai 0.0.1
    Uninstalling kai-0.0.1:
      Successfully uninstalled kai-0.0.1
  Running setup.py develop for kai

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [68]:
# Bring in the common deps
import os
import pprint

from langchain import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain

pp = pprint.PrettyPrinter(indent=2)


def get_java_in_result(text: str):
    _, after = text.split("```java")
    return after.split("```")[0]


def write_output_to_disk(
    out_dir,
    formatted_template,
    example_javaee_file_contents,
    example_quarkus_file_contents,
    result,
):
    try:
        os.makedirs(out_dir, exist_ok=True)
    except OSError as error:
        print(f"Error creating directory {out_dir}: {error}")
        raise error

    # Save the template to disk
    template_path = os.path.join(out_dir, "template.txt")
    with open(template_path, "w") as f:
        f.truncate()
        f.write(formatted_template)
    print(f"Saved template to {template_path}")

    # Save the result
    result_path = os.path.join(out_dir, "result.txt")
    with open(result_path, "w") as f:
        f.truncate()
        f.write(result)
    print(f"Saved result to {result_path}")

    # Extract the updated java code and save it
    updated_file_contents = get_java_in_result(result)
    updated_file_path = os.path.join(out_dir, "updated_file.java")
    with open(updated_file_path, "w") as f:
        f.truncate()
        f.write(updated_file_contents)
    print(f"Saved updated java file to {updated_file_path}")

    # Save the original source code
    original_file_path = os.path.join(out_dir, "original_file.java")
    with open(original_file_path, "w") as f:
        f.truncate()
        f.write(example_javaee_file_contents)
    print(f"Saved original java file to {original_file_path}")

    # Save the Quarkus version we already in Git to use as a comparison
    known_quarkus_file_path = os.path.join(out_dir, "quarkus_version_from_git.java")
    with open(known_quarkus_file_path, "w") as f:
        f.truncate()
        f.write(example_quarkus_file_contents)
    print(f"Saved Quarkus version from Git to {known_quarkus_file_path}")

# Create a Prompt
## Step #1:  Create a Prompt Template


In [69]:
template_with_solved_example_and_analysis = """
    # Java EE to Quarkus Migration
    You are an AI Assistant trained on migrating enterprise JavaEE code to Quarkus.
    I will give you an example of a JavaEE file and you will give me the Quarkus equivalent.

    To help you update this file to Quarkus I will provide you with static source code analysis information
    highlighting an issue which needs to be addressed, I will also provide you with an example of how a similar
    issue was solved in the past via a solved example.  You can refer to the solved example for a pattern of
    how to update the input Java EE file to Quarkus.

    Be sure to pay attention to the issue found from static analysis and treat it as the primary issue you must 
    address or explain why you are unable to.

    Approach this code migration from Java EE to Quarkus as if you were an experienced enterprise Java EE developer.
    Before attempting to migrate the code to Quarkus, explain each step of your reasoning through what changes 
    are required and why. 

    Pay attention to changes you make and impacts to external dependencies in the pom.xml as well as changes 
    to imports we need to consider.

    As you make changes that impact the pom.xml or imports, be sure you explain what needs to be updated.
    
    After you have shared your step by step thinking, provide a full output of the updated file:

    # Input information
    ## Issue found from static code analysis of the Java EE code which needs to be fixed to migrate to Quarkus
    Issue to fix:  "{analysis_message}"


    ## Solved Example Git Diff 
    This diff of the solved example shows what changes we made in past to address a similar problem. 
    Please note that this might have many solved examples that may be relevant to the issue found in the Java EE code. 
    Please consider this heavily in your response. 
    Iterate through the solved examples and consolidate the changes to the Java EE code to Quarkus.
    Display the consolidated changes in the diff format below.
    ```diff
    {solved_example_diff}
    ```

    ## Input file name
    Filename: "{src_file_name}"

    ## Input Line number of the issue first appearing in the Java EE code source code example below
    Line number: {analysis_lineNumber}
    
    ## Input source code file contents for "{src_file_name}"
    ```java 
    {src_file_contents}
    ```

    # Output Instructions
    Structure your ouput in Markdown format such as:

    ## Reasoning
    Write the step by step reasoning in this markdown section.
    If you are unsure of a step or reasoning, clearly state you are unsure and why.

    ## Updated File
    ```java
        Write the updated file for Quarkus in this section
    ```
   
    """

# Step 2: Gather the data we want to inject into the prompt

In [70]:
from kai.report import Report
from kai.scm import GitDiff
from kai.incident_store import IncidentStore

report_path = "../../samples/analysis_reports"
output_path = "../../samples/generated_output/incident_store"

incident_store = IncidentStore(report_path, output_path)
# We are limiting our experiment to a single rule for this run
ruleset_name = "custom-ruleset"
rule = "jms-to-reactive-quarkus-00010"

solved_example_diff = incident_store.get_solved_issue(ruleset_name, rule)
solved_diff = solved_example_diff[:2]
solved_diff = "\n".join(solved_diff)
print(solved_diff)


# Static code analysis was run prior and committed to this repo
path_helloworld_analysis = "./analysis_report/helloworld-mdb-quarkus/output.yaml"

helloworld_report = Report(path_helloworld_analysis)


helloworld_rule_data = helloworld_report.report[ruleset_name]["violations"][rule]

assert len(helloworld_rule_data["incidents"]), 2


helloworld_src_path = "../../samples/sample_repos/helloworld-mdb-quarkus"
helloworld_diff = GitDiff(helloworld_src_path)

# Resetting to 'main' for HelloWorld as that represents th initial state of the repo
helloworld_diff.checkout_branch("main")

# Now we can extract the info we need
## We will assume:
## .   HelloWorld will be our source application to migrate, so we will use it's 'main' branch which maps to Java EE
## .   CMT will serve as our solved example, so we will consult it's 'quarkus' branch

hw_entry = helloworld_rule_data["incidents"][0]
src_file_name = helloworld_report.get_cleaned_file_path(hw_entry["uri"])
src_file_contents = helloworld_diff.get_file_contents(src_file_name, "main")
analysis_message = hw_entry["message"]
analysis_lineNumber = hw_entry["lineNumber"]

Loading incident store from file: solved_incidents.yaml

Loaded incident store from file: ../../samples/generated_output/incident_store/solved_incidents.yaml

Found solved issues for custom-ruleset - jms-to-reactive-quarkus-00010 for app cmt
Found solved issues for custom-ruleset - jms-to-reactive-quarkus-00010 for app eap-coolstore-monolith
Found solved issues for custom-ruleset - jms-to-reactive-quarkus-00010 for app helloworld-mdb
@@ -16,14 +16,10 @@
  */
 package org.jboss.as.quickstarts.cmt.mdb;
 
-import java.util.logging.Logger;
+import org.eclipse.microprofile.reactive.messaging.Incoming;
 
-import javax.ejb.ActivationConfigProperty;
-import javax.ejb.MessageDriven;
-import javax.jms.JMSException;
-import javax.jms.Message;
-import javax.jms.MessageListener;
-import javax.jms.TextMessage;
+import io.quarkus.logging.Log;
+import jakarta.enterprise.context.ApplicationScoped;
 
 /**
  * <p>
@@ -33,28 +29,11 @@ import javax.jms.TextMessage;
  * @author Serge Pagop (spagop@redhat.co

# Step 3: Run the prompt through the LLM and write the output to disk

In [71]:
# For first run we will only include the source file and analysis info (we are omitting solved example)
template_args = {
    "src_file_name": src_file_name,
    "src_file_contents": src_file_contents,
    "analysis_message": analysis_message,
    "analysis_lineNumber": analysis_lineNumber,
    "solved_example_diff": solved_diff,
    # "solved_file_contents": solved_file_contents,
}
formatted_prompt = template_with_solved_example_and_analysis.format(**template_args)
# model_name = "gpt-4-1106-preview"
model_name = "gpt-3.5-turbo-16k"
llm = ChatOpenAI(temperature=0.1, model_name=model_name)
prompt = PromptTemplate.from_template(template_with_solved_example_and_analysis)
chain = LLMChain(llm=llm, prompt=prompt)
result = chain.run(template_args)

# For comparison, we will look up what the source file looks like from Quarkus branch
# this serves as a way of comparing to what the 'answer' is that was done manually by a knowledgeable developer
src_file_from_quarkus_branch = helloworld_diff.get_file_contents(
    src_file_name, "quarkus"
)

output_dir = f"output/{model_name}/helloworldmdb/{ruleset_name}/{rule}/few_shot/05-solved_examples/"
write_output_to_disk(
    output_dir,
    formatted_prompt,
    src_file_contents,
    src_file_from_quarkus_branch,
    result,
)

Saved template to output/gpt-3.5-turbo-16k/helloworldmdb/custom-ruleset/jms-to-reactive-quarkus-00010/few_shot/05-solved_examples/template.txt
Saved result to output/gpt-3.5-turbo-16k/helloworldmdb/custom-ruleset/jms-to-reactive-quarkus-00010/few_shot/05-solved_examples/result.txt
Saved updated java file to output/gpt-3.5-turbo-16k/helloworldmdb/custom-ruleset/jms-to-reactive-quarkus-00010/few_shot/05-solved_examples/updated_file.java
Saved original java file to output/gpt-3.5-turbo-16k/helloworldmdb/custom-ruleset/jms-to-reactive-quarkus-00010/few_shot/05-solved_examples/original_file.java
Saved Quarkus version from Git to output/gpt-3.5-turbo-16k/helloworldmdb/custom-ruleset/jms-to-reactive-quarkus-00010/few_shot/05-solved_examples/quarkus_version_from_git.java
